<a href="https://colab.research.google.com/github/paulodreher/data_science/blob/main/Machine_Learning_VNS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import copy
import random
import math
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer as freq
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook



# Settings


Loading Data

In [ ]:
data = pd.read_csv("/content/drive/My Drive/Pos/Machine Learning II/data.csv", delimiter=',')
df = pd.DataFrame(np.array(data))
y = data.iloc[:,-1]
x = data.iloc[:,:-1]

Normalizing

In [ ]:
  scaler = MinMaxScaler()
  x = scaler.fit_transform(x)

Binarizing

In [ ]:
def makeBinary(x):
  transformer = Binarizer().fit(x)
  return transformer.transform(x)

Preparing

In [ ]:
def preTrain(x,y):
 X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42)
 return np.array([X_train, X_test, y_train, y_test])


Trainning


In [ ]:
def setModel(s1,s2):
  return SVC(gamma=s1, C=s2)


In [ ]:
def trainning(settings):
  data,model = settings[0], settings[1]
  X_train, X_test, y_train, y_test = data[0], data[1], data[2], data[3]
  trained_model = model.fit(X_train, y_train)
  return np.array([trained_model,X_test,y_test])

Testing

In [ ]:
def testing(settings):
  trained_model, X_test, y_test = settings[0], settings[1], settings[2]
  y_predicted = trained_model.predict(X_test)
  return accuracy_score(y_test, y_predicted)


# VNS

Hyper Parameterizing

In [ ]:
def neighborhood(curvature, boundary):
  curvature_copy = copy.copy(curvature)
  boundary_copy = copy.copy(boundary)

  curvature += (2 * np.random.rand() - 1) * curvature
  boundary += (2 * np.random.rand() - 1) * boundary

  while curvature <= 0 or boundary <= 0:
    curvature = curvature_copy
    boundary = boundary_copy
    curvature += (2 * np.random.rand() - 1) * curvature
    boundary += (2 * np.random.rand() - 1) * boundary
  return curvature, boundary

In [ ]:
def initial_solution(settings):
  gamma, c = settings[4], settings[5]
  s1_model_set = setModel(gamma,c)
  s1_model_trained = trainning(np.array([settings,s1_model_set]))
  s1_model_assessed = testing(s1_model_trained)
  return s1_model_assessed

In [ ]:
def candidate_solution(settings):
  gamma, c = settings[4], settings[5]
  gamma_neighbor,c_neighbor = neighborhood(gamma,c)

  s2_model_set = setModel(gamma_neighbor,c_neighbor)
  s2_model_trained = trainning(np.array([settings,s2_model_set]))
  s2_model_assessed = testing(s2_model_trained)
  return np.array([s2_model_assessed,gamma_neighbor,c_neighbor])

In [ ]:
def local_search(settings):
    s1 = candidate_solution(settings)
    count = 0
    print(' ***********  LOCAL SEARCH ***********')
    while (count < 5):
      s2 = candidate_solution(settings)
      print('{}, {}: '.format(s1[0],s2[0]))
      if s2[0] > s1[0]:
        s1 = copy.deepcopy(s2)
      else:
        s2 = candidate_solution(settings)
        count = count + 1
    return s1[0]


VNS

In [ ]:
def vns(X,Y,neighborhood_size):
  kf = KFold(n_splits=10,shuffle=True)
  current_fold = 0
  for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    settings = np.array([X_train, X_test, y_train, y_test, 0.1, 5])
    s = initial_solution(settings)
    print('###########   FOLD: {}  ###########'.format(current_fold))
    for i in range(0,11):
      neighbor = 1
      print('\n')
      while neighbor < neighborhood_size:
        s2 = local_search(settings)
        print('\n')
        print('BEST LOCAL SEARCH {}: '.format(s2))
        print('\n')
        if (s2 > s):
          s = copy.deepcopy(s2)
          neighbor = 1
        else:
          neighbor = neighbor + 1

    print('VNS  BEST: {}: '.format(s))
    print('\n')
    current_fold = current_fold + 1
    #return s


  Results


In [ ]:
X = makeBinary(x)
vns(X,y,2)

###########   FOLD: 0  ###########


 ***********  LOCAL SEARCH ***********
0.42857142857142855, 0.42857142857142855: 
0.42857142857142855, 0.42857142857142855: 
0.42857142857142855, 0.42857142857142855: 
0.42857142857142855, 0.42857142857142855: 
0.42857142857142855, 0.42857142857142855: 


BEST LOCAL SEARCH 0.42857142857142855: 




 ***********  LOCAL SEARCH ***********
0.42857142857142855, 0.42857142857142855: 
0.42857142857142855, 0.42857142857142855: 
0.42857142857142855, 0.42857142857142855: 
0.42857142857142855, 0.42857142857142855: 
0.42857142857142855, 0.42857142857142855: 


BEST LOCAL SEARCH 0.42857142857142855: 




 ***********  LOCAL SEARCH ***********
0.42857142857142855, 0.42857142857142855: 
0.42857142857142855, 0.42857142857142855: 
0.42857142857142855, 0.42857142857142855: 
0.42857142857142855, 0.42857142857142855: 
0.42857142857142855, 0.42857142857142855: 


BEST LOCAL SEARCH 0.42857142857142855: 




 ***********  LOCAL SEARCH ***********
0.42857142857142855, 0.4